In [4]:
"""
Multi label XGBoost Best Parameter

step08_XGBClassifier_params.py -> 복사 & 붙여넣기 -> 수정 
"""
from xgboost import XGBClassifier # model
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_wine # 다항분류
from sklearn.datasets import make_blobs # [추가] 다항분류
from sklearn.metrics import accuracy_score, classification_report

#################################
## 1. XGBoost Hyper Parameter
#################################

# 1. dataset load : step05_XGBoost_test [참고]
X, y = make_blobs(n_samples=2000, n_features=4, centers=3, 
                   cluster_std=2.5, random_state=123)
X, y = load_wine(return_X_y=True)

In [5]:
# 2. dataset 생성 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
type(X_train) # numpy.ndarray


numpy.ndarray

In [6]:
# 3. model 생성
obj = XGBClassifier()
print(obj) # parameter 확인 
obj = XGBClassifier(colsample_bytree=1,
                    learning_rate=0.1, max_depth=3, 
                    min_child_weight=1,
                    n_estimators=200, 
                    objective="multi:softmax",
                    num_class=3)

# colsample_bytree=1 : 트리를 생성할때 훈련셋에서 feature 샘플링 비율(보통 :0.6~0.9)
# learning_rate=0.1 : 학습율(보통 : 0.01~0.2)
# max_depth=3 : tree 깊이, 과적합 영향
# min_child_weight=1 : 최소한의 자식 노드 가중치 합(자식 노드 분할 결정), 과적합 영향
# - 트리 분할 단계에서 min_child_weight 보다 더 적은 노드가 생성되면 트리 분할 멈춤
# n_estimators=100 : tree model 수 
# objective='binary:logistic' : 'reg:linear', 'multi:softmax'(num_class 지정)
# [해설] 예측력 향상 : n_estimators는 높이고, learning_rate은 낮춤
#        과적합 영향 : max_depth, min_child_weight  


XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              validate_parameters=None, verbosity=None)


In [7]:
##############################
## tree model 학습 조기종료 
##############################
evals = [(X_test, y_test)]
model = obj.fit(X_train, y_train, eval_metric='merror',
                early_stopping_rounds=100, eval_set=evals, verbose=True)
'''
eval_metric : tree model 성능평가 방법
 - 이항분류 : error, 멀티클래스 : merror, 회귀 : rmse
'''

[0]	validation_0-merror:0.05556
[1]	validation_0-merror:0.05556
[2]	validation_0-merror:0.05556
[3]	validation_0-merror:0.03704
[4]	validation_0-merror:0.03704
[5]	validation_0-merror:0.03704
[6]	validation_0-merror:0.03704
[7]	validation_0-merror:0.03704
[8]	validation_0-merror:0.03704
[9]	validation_0-merror:0.03704
[10]	validation_0-merror:0.03704
[11]	validation_0-merror:0.03704
[12]	validation_0-merror:0.01852
[13]	validation_0-merror:0.01852
[14]	validation_0-merror:0.01852
[15]	validation_0-merror:0.01852
[16]	validation_0-merror:0.01852
[17]	validation_0-merror:0.01852
[18]	validation_0-merror:0.01852
[19]	validation_0-merror:0.01852
[20]	validation_0-merror:0.01852
[21]	validation_0-merror:0.01852
[22]	validation_0-merror:0.01852
[23]	validation_0-merror:0.01852
[24]	validation_0-merror:0.01852
[25]	validation_0-merror:0.01852
[26]	validation_0-merror:0.01852
[27]	validation_0-merror:0.01852
[28]	validation_0-merror:0.01852
[29]	validation_0-merror:0.01852
[30]	validation_0-me

C:\Users\hyebin\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


'\n출력 결과 : 100번 tree model 학습과정에서 30번 이후 성능평가 지수가 최고  \nStopping. Best iteration:\n[30]    validation_0-merror:0.018519\n'

In [8]:
# 4. model 평가 
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print('accuracy =', acc) # accuracy = 0.944444444444444444

report = classification_report(y_test, y_pred)
print(report)


accuracy = 0.9814814814814815
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       1.00      0.95      0.98        21
           2       0.95      1.00      0.97        18

    accuracy                           0.98        54
   macro avg       0.98      0.98      0.98        54
weighted avg       0.98      0.98      0.98        54



In [9]:
#################################
## 2. Best Parameter
#################################
from sklearn.model_selection import GridSearchCV

# 기본 모델 객체 생성 
obj = XGBClassifier(objective="multi:softmax", num_class=3)

params = {'n_estimators':[100,200], 'max_depth':[3, 7], 
          'min_child_weight':[1,3],
          'colsample_bytree':[0.5, 0.7], 
          'learning_rate':[0.01, 0.5, 0.1]}


# cv와 iid 생략 : DeprecationWarning 발생(0.24 version에서 삭제 예정) 
gs = GridSearchCV(estimator=obj, param_grid=params, cv=3, iid=False)
# iid=False : 교차검정(폴드) 점수의 평균

model = gs.fit(X_train, y_train, eval_metric='merror', 
                eval_set=evals, verbose=True) 

print('best score =', model.best_score_)
print('best parameter =', model.best_params_)

[0]	validation_0-merror:0.05556
[1]	validation_0-merror:0.01852
[2]	validation_0-merror:0.01852
[3]	validation_0-merror:0.01852
[4]	validation_0-merror:0.01852
[5]	validation_0-merror:0.01852
[6]	validation_0-merror:0.01852
[7]	validation_0-merror:0.01852
[8]	validation_0-merror:0.01852
[9]	validation_0-merror:0.01852
[10]	validation_0-merror:0.01852
[11]	validation_0-merror:0.01852
[12]	validation_0-merror:0.01852
[13]	validation_0-merror:0.01852
[14]	validation_0-merror:0.01852
[15]	validation_0-merror:0.01852
[16]	validation_0-merror:0.01852
[17]	validation_0-merror:0.01852
[18]	validation_0-merror:0.01852
[19]	validation_0-merror:0.01852
[20]	validation_0-merror:0.01852
[21]	validation_0-merror:0.01852
[22]	validation_0-merror:0.01852
[23]	validation_0-merror:0.01852
[24]	validation_0-merror:0.01852
[25]	validation_0-merror:0.01852
[26]	validation_0-merror:0.01852
[27]	validation_0-merror:0.01852
[28]	validation_0-merror:0.01852
[29]	validation_0-merror:0.01852
[30]	validation_0-me

C:\Users\hyebin\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[54]	validation_0-merror:0.01852
[55]	validation_0-merror:0.01852
[56]	validation_0-merror:0.01852
[57]	validation_0-merror:0.01852
[58]	validation_0-merror:0.01852
[59]	validation_0-merror:0.01852
[60]	validation_0-merror:0.01852
[61]	validation_0-merror:0.01852
[62]	validation_0-merror:0.01852
[63]	validation_0-merror:0.01852
[64]	validation_0-merror:0.01852
[65]	validation_0-merror:0.01852
[66]	validation_0-merror:0.01852
[67]	validation_0-merror:0.01852
[68]	validation_0-merror:0.01852
[69]	validation_0-merror:0.01852
[70]	validation_0-merror:0.01852
[71]	validation_0-merror:0.01852
[72]	validation_0-merror:0.01852
[73]	validation_0-merror:0.01852
[74]	validation_0-merror:0.01852
[75]	validation_0-merror:0.01852
[76]	validation_0-merror:0.01852
[77]	validation_0-merror:0.01852
[78]	validation_0-merror:0.01852
[79]	validation_0-merror:0.01852
[80]	validation_0-merror:0.01852
[81]	validation_0-merror:0.01852
[82]	validation_0-merror:0.01852
[83]	validation_0-merror:0.01852
[84]	valid

[103]	validation_0-merror:0.01852
[104]	validation_0-merror:0.01852
[105]	validation_0-merror:0.01852
[106]	validation_0-merror:0.01852
[107]	validation_0-merror:0.01852
[108]	validation_0-merror:0.01852
[109]	validation_0-merror:0.01852
[110]	validation_0-merror:0.01852
[111]	validation_0-merror:0.01852
[112]	validation_0-merror:0.01852
[113]	validation_0-merror:0.01852
[114]	validation_0-merror:0.01852
[115]	validation_0-merror:0.01852
[116]	validation_0-merror:0.01852
[117]	validation_0-merror:0.01852
[118]	validation_0-merror:0.01852
[119]	validation_0-merror:0.01852
[120]	validation_0-merror:0.01852
[121]	validation_0-merror:0.01852
[122]	validation_0-merror:0.01852
[123]	validation_0-merror:0.01852
[124]	validation_0-merror:0.01852
[125]	validation_0-merror:0.01852
[126]	validation_0-merror:0.01852
[127]	validation_0-merror:0.01852
[128]	validation_0-merror:0.01852
[129]	validation_0-merror:0.01852
[130]	validation_0-merror:0.01852
[131]	validation_0-merror:0.01852
[132]	validati

[148]	validation_0-merror:0.01852
[149]	validation_0-merror:0.01852
[150]	validation_0-merror:0.01852
[151]	validation_0-merror:0.01852
[152]	validation_0-merror:0.01852
[153]	validation_0-merror:0.01852
[154]	validation_0-merror:0.01852
[155]	validation_0-merror:0.01852
[156]	validation_0-merror:0.01852
[157]	validation_0-merror:0.01852
[158]	validation_0-merror:0.01852
[159]	validation_0-merror:0.01852
[160]	validation_0-merror:0.01852
[161]	validation_0-merror:0.01852
[162]	validation_0-merror:0.01852
[163]	validation_0-merror:0.01852
[164]	validation_0-merror:0.01852
[165]	validation_0-merror:0.01852
[166]	validation_0-merror:0.01852
[167]	validation_0-merror:0.01852
[168]	validation_0-merror:0.01852
[169]	validation_0-merror:0.01852
[170]	validation_0-merror:0.01852
[171]	validation_0-merror:0.01852
[172]	validation_0-merror:0.01852
[173]	validation_0-merror:0.01852
[174]	validation_0-merror:0.01852
[175]	validation_0-merror:0.01852
[176]	validation_0-merror:0.01852
[177]	validati

...

[93]	validation_0-merror:0.01852
[94]	validation_0-merror:0.01852
[95]	validation_0-merror:0.01852
[96]	validation_0-merror:0.01852
[97]	validation_0-merror:0.01852
[98]	validation_0-merror:0.01852
[99]	validation_0-merror:0.01852
[100]	validation_0-merror:0.01852
[101]	validation_0-merror:0.01852
[102]	validation_0-merror:0.01852
[103]	validation_0-merror:0.01852
[104]	validation_0-merror:0.01852
[105]	validation_0-merror:0.01852
[106]	validation_0-merror:0.01852
[107]	validation_0-merror:0.01852
[108]	validation_0-merror:0.01852
[109]	validation_0-merror:0.01852
[110]	validation_0-merror:0.01852
[111]	validation_0-merror:0.01852
[112]	validation_0-merror:0.01852
[113]	validation_0-merror:0.01852
[114]	validation_0-merror:0.01852
[115]	validation_0-merror:0.01852
[116]	validation_0-merror:0.01852
[117]	validation_0-merror:0.01852
[118]	validation_0-merror:0.01852
[119]	validation_0-merror:0.01852
[120]	validation_0-merror:0.01852
[121]	validation_0-merror:0.01852
[122]	validation_0-me

[138]	validation_0-merror:0.01852
[139]	validation_0-merror:0.01852
[140]	validation_0-merror:0.01852
[141]	validation_0-merror:0.01852
[142]	validation_0-merror:0.01852
[143]	validation_0-merror:0.01852
[144]	validation_0-merror:0.01852
[145]	validation_0-merror:0.01852
[146]	validation_0-merror:0.01852
[147]	validation_0-merror:0.01852
[148]	validation_0-merror:0.01852
[149]	validation_0-merror:0.01852
[150]	validation_0-merror:0.01852
[151]	validation_0-merror:0.01852
[152]	validation_0-merror:0.01852
[153]	validation_0-merror:0.01852
[154]	validation_0-merror:0.01852
[155]	validation_0-merror:0.01852
[156]	validation_0-merror:0.01852
[157]	validation_0-merror:0.01852
[158]	validation_0-merror:0.01852
[159]	validation_0-merror:0.01852
[160]	validation_0-merror:0.01852
[161]	validation_0-merror:0.01852
[162]	validation_0-merror:0.01852
[163]	validation_0-merror:0.01852
[164]	validation_0-merror:0.01852
[165]	validation_0-merror:0.01852
[166]	validation_0-merror:0.01852
[167]	validati

[183]	validation_0-merror:0.01852
[184]	validation_0-merror:0.01852
[185]	validation_0-merror:0.01852

C:\Users\hyebin\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(



[186]	validation_0-merror:0.01852
[187]	validation_0-merror:0.01852
[188]	validation_0-merror:0.01852
[189]	validation_0-merror:0.01852
[190]	validation_0-merror:0.01852
[191]	validation_0-merror:0.01852
[192]	validation_0-merror:0.01852
[193]	validation_0-merror:0.01852
[194]	validation_0-merror:0.01852
[195]	validation_0-merror:0.01852
[196]	validation_0-merror:0.01852
[197]	validation_0-merror:0.01852
[198]	validation_0-merror:0.01852
[199]	validation_0-merror:0.01852
best score = 0.991869918699187
best parameter = {'colsample_bytree': 0.5, 'learning_rate': 0.01, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 200}
